In [ ]:
#imports
import numpy as np 
import copy as cp

#adds given value to total score
def add_score(sc, val):
    sc += val
    return sc

#move the grid to the left and update score
def move_left(grid, score):
    for i in range(4):
        non_zero = [x for x in grid[i,:] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[i,:] = np.array(non_zero + zero)
        for j in range(3):
            if grid[i,j] == grid[i,j+1]:
                grid[i,j] *= 2
                score = add_score(score, grid[i,j])
                grid[i,j+1] = 0
        non_zero = [x for x in grid[i,:] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[i,:] = np.array(non_zero + zero)
    return (grid,score)

#move the grid to the right and update score
def move_right(grid, score):
    for i in range(4):
        non_zero = [x for x in grid[i,:] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[i,:] = np.array(zero + non_zero[::-1])
        for j in range(3, 0, -1):
            if grid[i,j] == grid[i,j-1]:
                grid[i,j] *= 2
                score = add_score(score, grid[i,j])
                grid[i,j-1] = 0
        non_zero = [x for x in grid[i,:] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[i,:] = np.array(zero + non_zero[::-1])
    return (grid,score)

#move the grid up and update score
def move_up(grid, score):
    for i in range(4):
        non_zero = [x for x in grid[:,i] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[:,i] = np.array(non_zero + zero)
        for j in range(3):
            if grid[j,i] == grid[j+1,i]:
                grid[j,i] *= 2
                score = add_score(score, grid[j,i])
                grid[j+1,i] = 0
        non_zero = [x for x in grid[:,i] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[:,i] = np.array(non_zero + zero)
    return (grid,score)

#move the grid down and update score
def move_down(grid, score):
    for i in range(4):
        non_zero = [x for x in grid[:,i] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[:,i] = np.array(zero + non_zero[::-1])
        for j in range(3, 0, -1):
            if grid[j,i] == grid[j-1,i]:
                grid[j,i] *= 2
                score = add_score(score, grid[j,i])
                grid[j-1,i] = 0
        non_zero = [x for x in grid[:,i] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[:,i] = np.array(zero + non_zero[::-1])
    return (grid,score)

#generates new tile
def add_new_number(grid):
    zero_indices = np.where(grid == 0)
    if len(zero_indices[0]) == 0:
        return False
    index = np.random.choice(len(zero_indices[0]))
    i, j = zero_indices[0][index], zero_indices[1][index]
    grid[i,j] = 2 if np.random.random() < 0.9 else 4
    return True

#checks whether it is Game Over
def check_game_over(grid):
    
    if np.all(grid) == False:
        return False
    
    for row in range(4):
        for col in range(4):
            if row != 3:
                if (grid[row,col]==grid[row+1,col]):
                    return False
            if col != 3:
                if (grid[row,col]==grid[row,col+1]):
                    return False
            
    return True

#checks for potential win
def check_win(grid):
    return 2048 in grid

#move the grid in specified direction, check for win or lose
#raises RuntimeError "GO" if the game is in GAME OVER state
#raises RuntimeError "WIN" if the game is in WIN state
def play_2048(grid, move, score):
    
    orig_grid = cp.deepcopy(grid)
    
    if check_game_over(grid):
        raise RuntimeError("GO")
        
    if move == 'left':
        grid, score = move_left(grid, score)
    elif move == 'right':
        grid, score = move_right(grid, score)
    elif move == 'up':
        grid, score = move_up(grid, score)
    elif move == 'down':
        grid, score = move_down(grid, score)
    else:
        raise ValueError("Invalid move")
   
    if check_win(grid):
        raise RuntimeError("WIN")

    #check whether the move was possible
    if np.array_equal(grid,orig_grid) == False:
        add_new_number(grid)
    return (grid,score)

#starts a new game by generating two tiles and setting score to 0
def new_game():
    score = 0
    grid = np.zeros((4,4), dtype=int)
    add_new_number(grid)
    add_new_number(grid)
    
    return (grid, score)

#print of the grid
def print_grid(grid, score):
    print('Score: ', score)
    print("+----+----+----+----+")
    for i in range(4):
        line = "|"
        for j in range(4):
            if grid[i,j] == 0:
                line += "    |"
            else:
                line += "{:4d}|".format(grid[i,j])
        print(line)
        print("+----+----+----+----+")

In [ ]:
#Random direction solver
grid, score = new_game()
for i in range(1000):
    direction = np.random.choice(('left','right','up','down'))
    try:
        grid, score = play_2048(grid, direction, score)
    except RuntimeError as inst:
        if(str(inst)=="GO"):
            print("GAME OVER in ",(i+1)," moves")
        elif(str(inst)=="WIN"):
            print("WIN in ",(i+1)," moves")
        break
print_grid(grid, score)

In [ ]:
import math

#==================== Algorithms ====================
def evaluate(grid):
    empty = np.count_nonzero(grid == 0)
    max_tile = np.max(grid)
    return empty + (math.log2(max_tile) if max_tile > 0 else 0)

def get_valid_moves(grid):
    moves = []
    for move, func in zip(['left', 'right', 'up', 'down'], [move_left, move_right, move_up, move_down]):
        grid_copy = cp.deepcopy(grid)
        new_grid, _ = func(grid_copy, 0)
        if not np.array_equal(grid, new_grid):
            moves.append(move)
    return moves

# --- Alternating pairs ---
def alternating_pairs_move(grid):
    moves = ['right', 'down', 'left', 'up']
    for move in moves:
        if move in get_valid_moves(grid):
            return move
    return 'left'

# --- Spiral pattern ---
def spiral_pattern_move(grid):
    moves = ['up', 'left', 'down', 'right']
    for move in moves:
        if move in get_valid_moves(grid):
            return move
    return 'left'

# --- Expectimax ---
def expectimax(grid, depth, is_maximizing):
    if depth == 0 or check_game_over(grid):
        return evaluate(grid), None
    
    if is_maximizing:
        best_value = -float('inf')
        best_move = None
        for move in get_valid_moves(grid):
            grid_copy = cp.deepcopy(grid)
            new_grid, _ = play_2048(grid_copy, move, 0)
            value, _ = expectimax(new_grid, depth - 1, False)
            if value > best_value:
                best_value = value
                best_move = move
        return best_value, best_move
    else:
        empty = np.argwhere(grid == 0)
        if empty.size == 0:
            return evaluate(grid), None
        total_value = 0
        for (i, j) in empty[:min(4, len(empty))]:
            for tile, prob in [(2, 0.9), (4, 0.1)]:
                grid_copy = cp.deepcopy(grid)
                grid_copy[i, j] = tile
                value, _ = expectimax(grid_copy, depth - 1, True)
                total_value += prob * value
        return total_value / len(empty), None

def expectimax_move(grid):
    _, move = expectimax(grid, depth=3, is_maximizing=True)
    return move if move else get_valid_moves(grid)[0]

#==================== Stats ====================
def run_simulation(algo_func, num_games=30):
    stats = {
        'wins': 0,
        'losses': 0,
        'scores': [],
        'max_tiles': [],
        'moves': {'left': 0, 'right': 0, 'up': 0, 'down': 0, 'total': 0},
        'best_grid': None,
        'best_score': 0
    }

    for _ in range(num_games):
        grid, score = new_game()
        game_moves = {'left': 0, 'right': 0, 'up': 0, 'down': 0, 'total': 0}
        
        while not check_game_over(grid):
            move = algo_func(grid)
            game_moves[move] += 1
            game_moves['total'] += 1
            
            try:
                grid, gained = play_2048(grid, move, 0)
                score += gained
            except RuntimeError as e:
                if str(e) == "WIN":
                    stats['wins'] += 1
                elif str(e) == "GO":
                    stats['losses'] += 1
                break

        stats['scores'].append(score)
        stats['max_tiles'].append(np.max(grid))
        
        if score > stats['best_score']:
            stats['best_score'] = score
            stats['best_grid'] = cp.deepcopy(grid)

        for m in ['left', 'right', 'up', 'down', 'total']:
            stats['moves'][m] += game_moves[m]

    return stats

def print_stats(stats):
    num_games = len(stats['scores'])
    avg_score = sum(stats['scores']) / num_games
    avg_max_tile = sum(stats['max_tiles']) / num_games
    avg_moves = {m: stats['moves'][m] / num_games for m in stats['moves']}
    
    print(f"  Odehraných her: {num_games}")
    print(f"  Výhry: {stats['wins']}, Prohry: {stats['losses']}")
    print(f"  Nejlepší skóre: {stats['best_score']}")
    print(f"  Průměrné skóre: {avg_score:.2f}")
    print(f"  Průměrná dosažená maximální buňka: {avg_max_tile:.2f}")
    
    print("  Průměrný počet tahů na hru:")
    for m in ['left', 'right', 'up', 'down', 'total']:
        print(f"    {m}: {avg_moves[m]:.2f}")

    print("\nNejlepší dosažená hra:\n")
    print_grid(stats['best_grid'], stats['best_score'])
    print("\n")

#==================== Main Function ====================
if __name__ == "__main__":
    algorithms = [
        ("Alternating pairs", alternating_pairs_move),
        ("Spiral Pattern", spiral_pattern_move),
        ("Expectimax", expectimax_move)
    ]
    
    for name, algo in algorithms:
        print(f"Spouštím algoritmus: {name}")
        stats = run_simulation(algo, num_games=30)
        print_stats(stats)


Spouštím algoritmus: Alternating pairs
  Odehraných her: 30
  Výhry: 0, Prohry: 0
  Nejlepší skóre: 6656
  Průměrné skóre: 1739.06
  Průměrná dosažená maximální buňka: 143.56
  Průměrný počet tahů na hru:
    left: 7.65
    right: 124.86
    up: 0.01
    down: 35.10
    total: 167.62

Nejlepší dosažená hra:

Score:  6656
+----+----+----+----+
|   2|   4|  16|   4|
+----+----+----+----+
|   4|  16|   2|   8|
+----+----+----+----+
|   8|  32|  64|   4|
+----+----+----+----+
|  16|  64| 256| 512|
+----+----+----+----+



Spouštím algoritmus: Spiral Pattern
  Odehraných her: 30
  Výhry: 0, Prohry: 0
  Nejlepší skóre: 6768
  Průměrné skóre: 2316.47
  Průměrná dosažená maximální buňka: 186.22
  Průměrný počet tahů na hru:
    left: 43.04
    right: 0.02
    up: 156.28
    down: 9.54
    total: 208.88

Nejlepší dosažená hra:

Score:  6768
+----+----+----+----+
| 512| 128|  64|   2|
+----+----+----+----+
| 128|  64|  32|   8|
+----+----+----+----+
|  64|  16|   8|   4|
+----+----+----+----+
|  32|   8|   4|   2|
+----+----+----+----+



Spouštím algoritmus: Expectimax
  Odehraných her: 30
  Výhry: 0, Prohry: 0
  Nejlepší skóre: 15336
  Průměrné skóre: 4782.28
  Průměrná dosažená maximální buňka: 398.00
  Průměrný počet tahů na hru:
    left: 94.42
    right: 81.59
    up: 89.85
    down: 79.74
    total: 345.60

Nejlepší dosažená hra:

Score:  15336
+----+----+----+----+
|   2|   8|   2|   4|
+----+----+----+----+
|   4|  64|  32|   2|
+----+----+----+----+
| 256|  16| 512|  16|
+----+----+----+----+
|   4|   8|1024|   2|
+----+----+----+----+
